# HSC Catalog Cuts

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import h5py as h
import glob
from astropy.table import Table, vstack

In [2]:
import pandas as pd

In [4]:
hscdir = '/global/cfs/cdirs/lsst/groups/LSS/HSC_data/'

In [5]:
def getdata(filename):
    hsc_shear = Table.read(hscdir + filename, memmap=True) 
    return hsc_shear

In [6]:
hscfiles = ['HSC_WIDE_GAMA09H.fits',
             'HSC_WIDE_GAMA15H.fits', 'HSC_WIDE_HECTOMAP.fits',
             'HSC_WIDE_VVDS_part1.fits','HSC_WIDE_VVDS_part2.fits',
             'HSC_WIDE_WIDE12H.fits','HSC_WIDE_XMM.fits']

In [7]:
tables = []
print('loading data')
for i in range(len(hscfiles)):
    d = getdata(hscfiles[i])
    tables.append(d)
print('loaded data')
#Joining the tables
hsc_shearall = vstack(tables)

loading data
loaded data


In [10]:
for col in hsc_shearall.columns:
    print(col)

object_id
object_id_isnull
ra
ra_isnull
dec
dec_isnull
tract
tract_isnull
patch
patch_isnull
merge_peak_g
merge_peak_g_isnull
merge_peak_r
merge_peak_r_isnull
merge_peak_i
merge_peak_i_isnull
merge_peak_z
merge_peak_z_isnull
merge_peak_y
merge_peak_y_isnull
gcountinputs
gcountinputs_isnull
rcountinputs
rcountinputs_isnull
icountinputs
icountinputs_isnull
zcountinputs
zcountinputs_isnull
ycountinputs
ycountinputs_isnull
iflags_pixel_bright_object_center
iflags_pixel_bright_object_center_isnull
iflags_pixel_bright_object_any
iflags_pixel_bright_object_any_isnull
iclassification_extendedness
iclassification_extendedness_isnull
iblendedness_abs_flux
iblendedness_abs_flux_isnull
a_g
a_g_isnull
a_r
a_r_isnull
a_i
a_i_isnull
a_z
a_z_isnull
a_y
a_y_isnull
gcmodel_flux
gcmodel_flux_isnull
rcmodel_flux
rcmodel_flux_isnull
icmodel_flux
icmodel_flux_isnull
zcmodel_flux
zcmodel_flux_isnull
ycmodel_flux
ycmodel_flux_isnull
gcmodel_flux_err
gcmodel_flux_err_isnull
rcmodel_flux_err
rcmodel_flux_err_is

In [3]:
df = pd.read_pickle('/global/cscratch1/sd/elp25/txpipe-reanalysis/hsc/data/hsc_cat.pkl')

In [4]:
#is_primary = (df['idetect_is_primary']==True)
#deblend = (df['ideblend_skipped'] ==False)
#bad_centroid = (df['iflags_badcentroid']==False)
#centroid_flag = (df['centroid_sdss_flags']==False)
#pixel_edge = (df['iflags_pixel_edge']==False)
#pixel_interpolated = (df['iflags_pixel_interpolated_center']==False)
#pixel_saturated = (df['iflags_pixel_saturated_center']==False)
#pixel_cr = (df['iflags_pixel_cr_center']==False)
#pixel_bad = (df['flags_pixel_bad']==False)
#pixel_suspect = (df['iflags_pixel_suspect_center']==False)
#pixel_clipped = (df['flags_pixel_clipped_any']==False)

In [5]:
regauss_flag = (df['ishape_hsm_regauss_flags']==False)
print('regauss_flags')
regaus_sigma_nan = (df['ishape_hsm_regauss_sigma'] !=np.nan)
print('regauss_sigma')
extendedness = (df['iclassification_extendedness']==1)
print('extendedness')
flux_cmodel = (df['icmodel_flux']/df['icmodel_flux_err']>=10)
print('flux')
regauss_resolution = (df['ishape_hsm_regauss_resolution']>=0.3)
print('regauss_resolution')
regauss_e = ((df['ishape_hsm_regauss_e1']**2+df['ishape_hsm_regauss_e2']**2)**(1/2)<2)
print('regauss_e1')
regauss_sigma_cut1 = (0<=df['ishape_hsm_regauss_sigma'])
regauss_sigma_cut2 = (df['ishape_hsm_regauss_sigma']<=0.4)
print('sigma_cut')
imag_cut = (df['icmodel_mag']-df['a_i']<=24.5)
#print('mag_cmodel')
blendedness_abs_flux = (df['iblendedness_abs_flux']< 10**(-0.375))
print('blendedness')
gflux_cut = (df['icmodel_flux']/df['gcmodel_flux_err']>=5)
rflux_cut = (df['rcmodel_flux']/df['rcmodel_flux_err']>=5)
zflux_cut = (df['zcmodel_flux']/df['zcmodel_flux_err']>=5)
yflux_cut = (df['ycmodel_flux']/df['ycmodel_flux_err']>=5)
print('fluxes')

regauss_flags
regauss_sigma
extendedness
flux
regauss_resolution
regauss_e1
sigma_cut
blendedness
fluxes


In [6]:
all_cuts = (regauss_flag) & (regaus_sigma_nan) & (extendedness) & (flux_cmodel) & (regauss_resolution) & (regauss_e) & (regauss_sigma_cut1) & (regauss_sigma_cut2) & (blendedness_abs_flux) & (gflux_cut) & (rflux_cut) & (zflux_cut) & (yflux_cut) & (imag_cut)

In [8]:
len(df[all_cuts])

20711894

In [53]:
np.unique(df['iclassification_extendedness'])

array([ 0.,  1., nan, ..., nan, nan, nan], dtype=float32)

In [9]:
ishape_hsm_regauss_e1_isnull = df['ishape_hsm_regauss_e1_isnull'] == False
ishape_hsm_regauss_e2_isnull = df['ishape_hsm_regauss_e2_isnull'] == False
ishape_hsm_regauss_derived_shape_weight_isnull = df['ishape_hsm_regauss_derived_shape_weight_isnull'] == False 
ishape_hsm_regauss_derived_shear_bias_m_isnull = df['ishape_hsm_regauss_derived_shear_bias_m_isnull'] == False 
ishape_hsm_regauss_derived_shear_bias_c1_isnull = df['ishape_hsm_regauss_derived_shear_bias_c1_isnull'] == False 
ishape_hsm_regauss_derived_shear_bias_c2_isnull = df['ishape_hsm_regauss_derived_shear_bias_c2_isnull'] == False 
ishape_hsm_regauss_derived_sigma_e_isnull = df['ishape_hsm_regauss_derived_sigma_e_isnull'] == False 

In [11]:
null_cuts = (ishape_hsm_regauss_e1_isnull) & (ishape_hsm_regauss_e2_isnull) & (ishape_hsm_regauss_derived_shape_weight_isnull) & (ishape_hsm_regauss_derived_shear_bias_m_isnull) & (ishape_hsm_regauss_derived_shear_bias_c1_isnull) & (ishape_hsm_regauss_derived_shear_bias_c2_isnull) &  (ishape_hsm_regauss_derived_sigma_e_isnull) 

Objects reamining: 22,992,773

In [12]:
len(df[all_cuts&null_cuts])

10613947

In [51]:
#10,707,132